In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.signal import resample
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
import tensorflow as tf
import random

In [3]:
train = pd.read_csv('../data/sensor_train.csv')
test = pd.read_csv('../data/sensor_test.csv')
sub = pd.read_csv('../data/提交结果示例.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [4]:
train.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0


In [5]:
train['mod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['modg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
test['mod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['modg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5

In [6]:
x = np.zeros((7292, 60, 8))
t = np.zeros((7500, 60, 8))
for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:60]
    x[i,:,:] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],
                                    axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i,:,:] = resample(tmp.drop(['fragment_id', 'time_point'],
                                    axis=1), 60, np.array(tmp.time_point))[0]

100%|██████████| 7500/7500 [00:11<00:00, 666.83it/s]


In [7]:
def Net():
    input = Input(shape=(60, 8))
    X = Conv1D(filters=64,
               kernel_size=(3),
               activation='relu',
               padding='same')(input)
    X = Conv1D(filters=128,
               kernel_size=(3),
               activation='relu',
               padding='same')(X)
    X = MaxPooling1D()(X)
    X = Conv1D(filters=256,
               kernel_size=(3),
               activation='relu',
               padding='same')(X)
    X = Conv1D(filters=512,
               kernel_size=(3),
               activation='relu',
               padding='same')(X)
    X = GlobalMaxPooling1D()(X)
    X = Dropout(0.3)(X)
    X = Dense(19, activation='softmax')(X)
    return Model([input], X)

In [8]:
kfold = StratifiedKFold(5, shuffle=True)

In [9]:
# proba_t = np.zeros((7500, 19))
# for fold, (xx, yy) in enumerate(kfold.split(x, y)):
#     y_ = to_categorical(y, num_classes=19)
    
    
#     model = Net()
#     model.compile(loss='categorical_crossentropy',
#                   optimizer=Adam(),
#                   metrics=['acc'])
    
#     # 验证集acc不增加，自动减少学习率
#     plateau = ReduceLROnPlateau(monitor="val_acc",
#                                 verbose=0,
#                                 mode='max',
#                                 factor=0.1,
#                                 patience=6)

#     # 防止过拟合，在验证集最优保存模型
#     early_stopping = EarlyStopping(monitor='val_acc',
#                                    verbose=0,
#                                    mode='max',
#                                    patience=10)

#     # 保存
#     checkpoint = ModelCheckpoint(f'fold{fold}.h5',
#                                  monitor='val_acc',
#                                  verbose=0,
#                                  mode='max',
#                                  save_best_only=True)
#     model.fit(x[xx], y_[xx],
#               epochs=100,
#               batch_size=512,
#               verbose=1,
#               shuffle=True,
#               validation_data=(x[yy], y_[yy]),
#               callbacks=[plateau, early_stopping, checkpoint])
#     model.load_weights(f'fold{fold}.h5')
#     proba_t += model.predict(t, verbose=0, batch_size=1024) / 5.

In [10]:
proba_t = np.zeros((7500, 19))
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    y_ = to_categorical(y, num_classes=19)
    break

In [11]:
model = Net()
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])

In [12]:
def shuffle_index_fun(index):
    random.shuffle(index)
    return index.copy()

In [13]:
def shuffle_data(data_x, data_y):
    shuffle_index = list(range(data_x.shape[0]))
    r_index = shuffle_index_fun(shuffle_index)
    return data_x[r_index], data_y[r_index]

In [14]:
def mixup(batch_x, batch_y, random_max=0.02):
    shuffle_index = list(range(batch_x.shape[0]))
    r_index1 = shuffle_index_fun(shuffle_index)
    r_index2 = shuffle_index_fun(shuffle_index)
    
    rd = random.uniform(0, random_max)
    batch_x_mixup = (1-rd) * batch_x[r_index1] + rd * batch_x[r_index2]
    batch_y_mixup = (1-rd) * batch_y[r_index1] + rd * batch_y[r_index2]
    return batch_x_mixup, batch_y_mixup

In [15]:
def noise(batch_x, batch_y, random_max=0.02):
    size = batch_x.shape
    
    batch_x_noise = batch_x + np.random.uniform(-random_max,random_max,size=size)
    batch_y_noise = batch_y
    return batch_x_noise, batch_y_noise

# 迭代器 产生

In [16]:
def generator(data_x, data_y, batch_size, dataAugment, shuffle=False):
    
    # 先shuffle打乱
    if shuffle == True:
        data_x, data_y = shuffle_data(data_x, data_y)
    
    # 产生batch
    while True:
        data_num = data_x.shape[0]
        range_num = data_num // batch_size + 1
        for i in range(range_num):
            begin_index, end_index = i * batch_size, (i + 1) * batch_size

            list_batch_x, list_batch_y = [], []
            if begin_index < data_num:
                batch_x, batch_y = data_x[begin_index:end_index], data_y[begin_index:end_index]
                list_batch_x.append(batch_x), list_batch_y.append(batch_y)

                # 数据增强 
                if dataAugment == True:
                    # 数据增强 mixup
                    batch_x_mixup, batch_y_mixup = mixup(batch_x, batch_y, random_max=0.05)
                    list_batch_x.append(batch_x_mixup), list_batch_y.append(batch_y_mixup)

                    # 数据增强 噪声
                    batch_x_noise, batch_y_noise = noise(batch_x, batch_y, random_max=0.01)
                    list_batch_x.append(batch_x_noise), list_batch_y.append(batch_y_noise)

                    # 其他

                batch_x_yield, batch_y_yield = np.vstack(list_batch_x), np.vstack(list_batch_y)
                yield batch_x_yield, batch_y_yield

# 保证数据一致

### 打乱

In [17]:
x[xx], y_[xx] = shuffle_data(x[xx], y_[xx])
batch_size = 512
steps_per_epoch = x[xx].shape[0]//batch_size+1

### 数据增强

In [18]:
ge = generator(x[xx], y_[xx], batch_size, True)

### 不数据增强

In [19]:
ge_false = generator(x[xx], y_[xx], batch_size, False)

# 数据增强测试

### 不增强

In [20]:
model = Net()
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])

In [21]:
model.fit_generator(ge_false, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

12/12 [==============================] - 16s 1s/step - loss: 2.5836 - acc: 0.1972 - val_loss: 2.1501 - val_acc: 0.3112


### 增强

In [22]:
model = Net()
model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(),
                  metrics=['acc'])

In [23]:
model.fit_generator(ge, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

12/12 [==============================] - 38s 3s/step - loss: 2.6717 - acc: 0.1797 - val_loss: 2.2555 - val_acc: 0.2940


# 自定义 loss

In [24]:
def my_loss(y_true, y_pred, smooth_wight=0.0000001, eps=0.00001):
    
    # sigmod
    loss = (y_true - smooth_wight) * tf.math.log(y_pred + eps) + (1 - y_true - smooth_wight) * tf.math.log(1 - y_pred + eps)
    loss = -tf.reduce_mean(loss)
    
    # loss = tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)))
    # loss = -tf.reduce_mean(y_true*tf.math.log(y_pred + eps))
    # loss = tf.losses.categorical_crossentropy(y_true, y_pred,label_smoothing=smooth_wight)
    
    return loss     

In [25]:
model = Net()
model.compile(loss=my_loss,
                  optimizer=Adam(),
                  metrics=['acc'])

In [26]:
model.fit_generator(ge_false, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

12/12 [==============================] - 15s 1s/step - loss: 0.1843 - acc: 0.2225 - val_loss: 0.1634 - val_acc: 0.3372


# 查看代码

In [27]:
# tf.losses.categorical_crossentropy?

In [28]:
# !cat ~/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/losses.py

# metrics分数

In [29]:
def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0

In [30]:
# 场景+动作

In [31]:
def acc_combo(y_true, y_pred):
    try:
        # 场景
        map_scene = tf.constant(['A','A','A','A','D','A','B','B','B','B','B','A','C','C','C','B','C','C','C'])
        # 动作
        map_action = tf.constant(['0','1','2','3','4','5','1','5','2','3','0','6','1','3','0','6','2','5','6'])

        # 预测标签
        # 动作+场景
        y_true_index, y_pred_index = tf.argmax(y_true,axis=1), tf.argmax(y_pred,axis=1)
        # 动作
        y_true_index_scene, y_pred_index_scene = tf.nn.embedding_lookup(map_scene,y_true_index), tf.nn.embedding_lookup(map_action,y_pred_index)
        # 场景
        y_true_index_action, y_pred_index_action = tf.nn.embedding_lookup(map_action,y_true_index), tf.nn.embedding_lookup(map_action,y_pred_index)

        # 动作 1.0/7
        acc_scene = tf.cast(tf.equal(y_true_index_scene, y_pred_index_scene),tf.float16)
        # 场景 1.0/3 
        acc_action = tf.cast(tf.equal(y_true_index_action, y_pred_index_action),tf.float16)
        # 动作 + 场景 (1-1.0/3-1.0/7) = 11/21
        acc_classification = tf.cast(tf.equal(y_true_index, y_pred_index),tf.float16)

        # 得分
        score = tf.reduce_mean(1.0/7 * acc_scene + 1.0/3 * acc_action + 11/21 * acc_classification)
    except:
        print(y_true), print(y_pred)
        print(y_true_index), print(y_pred_index)
    return score 

In [32]:
model = Net()
model.compile(loss=my_loss,
                  optimizer=Adam(),
                  metrics=[acc_combo,'acc'])

In [33]:
model.fit_generator(ge_false, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

12/12 [==============================] - 14s 1s/step - loss: 0.1928 - acc_combo: 0.2346 - acc: 0.1946 - val_loss: 0.1674 - val_acc_combo: 0.3375 - val_acc: 0.3098


# 直接优化可以吗？

In [34]:
model = Net()
model.compile(loss=acc_combo,
                  optimizer=Adam(),
                  metrics=[acc_combo])

In [35]:
model.fit_generator(ge_false, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

ValueError: No gradients provided for any variable: ['conv1d_20/kernel:0', 'conv1d_20/bias:0', 'conv1d_21/kernel:0', 'conv1d_21/bias:0', 'conv1d_22/kernel:0', 'conv1d_22/bias:0', 'conv1d_23/kernel:0', 'conv1d_23/bias:0', 'dense_5/kernel:0', 'dense_5/bias:0'].

# 测试加权

In [36]:
def acc_combo_loss(y_true, y_pred, smooth_wight=0.001, eps=0.001):

    # 场景
    map_scene = tf.constant(['A','A','A','A','D','A','B','B','B','B','B','A','C','C','C','B','C','C','C'])
    # 动作
    map_action = tf.constant(['0','1','2','3','4','5','1','5','2','3','0','6','1','3','0','6','2','5','6'])

    # 预测标签
    # 动作+场景
    y_true_index, y_pred_index = tf.argmax(y_true,axis=1), tf.argmax(y_pred,axis=1)
    # 动作
    y_true_index_scene, y_pred_index_scene = tf.nn.embedding_lookup(map_scene,y_true_index), tf.nn.embedding_lookup(map_action,y_pred_index)
    # 场景
    y_true_index_action, y_pred_index_action = tf.nn.embedding_lookup(map_action,y_true_index), tf.nn.embedding_lookup(map_action,y_pred_index)

    # 动作 1.0/7
    acc_scene = tf.cast(tf.equal(y_true_index_scene, y_pred_index_scene),tf.float32)
    # 场景 1.0/3 
    acc_action = tf.cast(tf.equal(y_true_index_action, y_pred_index_action),tf.float32)
    # 动作 + 场景 (1-1.0/3-1.0/7) = 11/21
    acc_classification = tf.cast(tf.equal(y_true_index, y_pred_index),tf.float32)

    # 得分
    score = tf.reduce_mean(1.0/7 * acc_scene + 1.0/3 * acc_action + 11/21 * acc_classification)
    loss = (y_true - smooth_wight) * tf.math.log(y_pred + eps) + (1 - y_true - smooth_wight) * tf.math.log(1 - y_pred + eps)
#     loss = - tf.reduce_mean(loss)
    loss = - tf.reduce_sum(loss ** (1+ 0.1 * score))
    return loss

In [37]:
def my_bias_loss(y_true, y_pred, smooth_wight=0.001, eps=0.001):

    loss = (y_true - smooth_wight) * tf.math.log(y_pred + eps)  +\
    (1 - y_true - smooth_wight) * tf.math.log(1 - y_pred + eps) 
    loss = loss * (y_true - y_pred) ** 2
    loss = -tf.reduce_mean(loss) 

    return loss

# 测试对比 score加入

In [38]:
model = Net()
model.compile(loss=my_bias_loss,
                  optimizer=Adam(),
                  metrics=[acc_combo,'acc'])

In [39]:
# model.fit(x[xx], y_[xx], epochs=1, batch_size=batch_size)

In [40]:
model.fit_generator(ge_false, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

12/12 [==============================] - 14s 1s/step - loss: 0.1225 - acc_combo: 0.2374 - acc: 0.1915 - val_loss: 0.0971 - val_acc_combo: 0.3309 - val_acc: 0.2858


# 测试对比 不加入

In [41]:
model = Net()
model.compile(loss=my_loss,
                  optimizer=Adam(),
                  metrics=[acc_combo,'acc'])

In [42]:
# model.fit(x[xx], y_[xx], epochs=1, batch_size=batch_size,  validation_data=(x[yy], y_[yy]))

In [43]:
model.fit_generator(ge_false, epochs=1, steps_per_epoch=steps_per_epoch, validation_data=(x[yy], y_[yy]))

12/12 [==============================] - 15s 1s/step - loss: 0.1851 - acc_combo: 0.2525 - acc: 0.2100 - val_loss: 0.1642 - val_acc_combo: 0.3316 - val_acc: 0.2879


# 有没有问题？

In [44]:
set(['A','A','A','A','D','A','B','B','B','B','B','A','C','C','C','B','C','C','C'])

{'A', 'B', 'C', 'D'}

In [45]:
set(['0','1','2','3','4','5','1','5','2','3','0','6','1','3','0','6','2','5','6'])

{'0', '1', '2', '3', '4', '5', '6'}

In [46]:
def Net2():
    input = Input(shape=(60, 8))
    X = Conv1D(filters=64,
               kernel_size=(3),
               activation='relu',
               padding='same')(input)
    X = Conv1D(filters=128,
               kernel_size=(3),
               activation='relu',
               padding='same')(X)
    X = MaxPooling1D()(X)
    X = Conv1D(filters=256,
               kernel_size=(3),
               activation='relu',
               padding='same')(X)
    X = Conv1D(filters=512,
               kernel_size=(3),
               activation='relu',
               padding='same')(X)
    X = GlobalMaxPooling1D()(X)
    X = Dropout(0.3)(X)
    X_cls = Dense(19, activation='softmax')(X)
    X_sence = Dense(4, activation='softmax')(X)
    X_action = Dense(7, activation='softmax')(X)
    
    return Model([input], [X_cls,X_sence,X_action])

In [47]:
def loss_multi(y_true, y_pred, smooth_wight=0.001, eps=0.001):
    
    X_cls, X_sence, X_action = y_pred[0], y_pred[1], y_pred[2]
    y_cls, y_sence, y_action = y_true[0], y_true[1], y_true[2]
    
    loss_cls = tf.losses.categorical_crossentropy(y_cls, X_cls)
    loss_sence = tf.losses.categorical_crossentropy(y_sence, X_sence)
    loss_action = tf.losses.categorical_crossentropy(y_action, X_action)
    
#     return [loss_cls, loss_sence, loss_action]
    return tf.reduce_mean(1.0/7 * loss_sence + 1.0/3 * loss_action + 11/21 * loss_cls)

In [48]:
model2 = Net2()
model2.compile(loss=loss_multi,
                  optimizer=Adam(),
                  metrics=['acc'])

In [49]:
model2.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 60, 8)]      0                                            
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 60, 64)       1600        input_9[0][0]                    
__________________________________________________________________________________________________
conv1d_33 (Conv1D)              (None, 60, 128)      24704       conv1d_32[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_8 (MaxPooling1D)  (None, 30, 128)      0           conv1d_33[0][0]                  
____________________________________________________________________________________________

# 多个loss

In [53]:
mapping = {0: 'A_0', 1: 'A_1', 2: 'A_2', 3: 'A_3', 
        4: 'D_4', 5: 'A_5', 6: 'B_1',7: 'B_5', 
        8: 'B_2', 9: 'B_3', 10: 'B_0', 11: 'A_6', 
        12: 'C_1', 13: 'C_3', 14: 'C_0', 15: 'B_6', 
        16: 'C_2', 17: 'C_5', 18: 'C_6'}

In [54]:
mapping3 = {'A':0, 'B':1, 'C':2, 'D':2}

In [55]:
train = pd.read_csv('../data/sensor_train.csv')
test = pd.read_csv('../data/sensor_test.csv')
sub = pd.read_csv('../data/提交结果示例.csv')
train['sence'] = train['behavior_id'].map(lambda x:mapping3[mapping[x].split('_')[0]])
train['action'] = train['behavior_id'].map(lambda x:int(mapping[x].split('_')[1]))
y_cls = train.groupby('fragment_id')['behavior_id'].min()
y_scene = train.groupby('fragment_id')['sence'].min()
y_action = train.groupby('fragment_id')['action'].min()

In [56]:
train.head()

,fragment_id,time_point,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg,behavior_id,sence,action
0,0,27,0.3,-0.3,0.1,0.6,4.5,8.8,0,0,0
1,0,108,0.1,-0.0,-0.4,0.4,4.7,8.4,0,0,0
2,0,198,0.1,0.0,0.3,0.9,4.6,9.0,0,0,0
3,0,297,0.1,-0.1,-0.5,0.8,4.7,7.2,0,0,0
4,0,388,0.1,0.2,0.6,0.9,4.7,8.9,0,0,0


In [57]:
train['mod'] = (train.acc_x ** 2 + train.acc_y ** 2 + train.acc_z ** 2) ** .5
train['modg'] = (train.acc_xg ** 2 + train.acc_yg ** 2 + train.acc_zg ** 2) ** .5
test['mod'] = (test.acc_x ** 2 + test.acc_y ** 2 + test.acc_z ** 2) ** .5
test['modg'] = (test.acc_xg ** 2 + test.acc_yg ** 2 + test.acc_zg ** 2) ** .5

In [58]:
x = np.zeros((7292, 60, 8))
t = np.zeros((7500, 60, 8))
for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:60]
    x[i,:,:] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id', 'sence', 'action'],
                                    axis=1), 60, np.array(tmp.time_point))[0]
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:60]
    t[i,:,:] = resample(tmp.drop(['fragment_id', 'time_point'],
                                    axis=1), 60, np.array(tmp.time_point))[0]

100%|██████████| 7500/7500 [00:11<00:00, 628.31it/s]


In [69]:
y_cls_ = to_categorical(y_cls, num_classes=19)
y_sence_ = to_categorical(y_scene, num_classes=4)
y_action_ = to_categorical(y_action, num_classes=7)

In [75]:
for fold, (xx, yy) in enumerate(kfold.split(x, y)):
    break

In [76]:
model2.fit(x[xx],[y_cls_[xx],y_sence_[xx],y_action_[xx]],\
           validation_data=[x[yy], [y_cls_[yy],y_sence_[yy],y_action_[yy]]])

Train on 5833 samples, validate on 1459 samples
5833/5833 [==============================] - 13s 2ms/sample - loss: 5.0431 - dense_8_loss: 2.5018 - dense_9_loss: 1.0243 - dense_10_loss: 1.5131 - dense_8_acc: 0.2146 - dense_9_acc: 0.4571 - dense_10_acc: 0.4751 - val_loss: 4.8070 - val_dense_8_loss: 2.3757 - val_dense_9_loss: 1.0768 - val_dense_10_loss: 1.3940 - val_dense_8_acc: 0.1960 - val_dense_9_acc: 0.4202 - val_dense_10_acc: 0.4770


# Stacking

In [ ]:
# -*- coding: utf-8 -*-
###########################################################
# 注意提交前删除
import pandas as pd 
model_sample = pd.read_csv("model_sample.csv")
verify_sample = pd.read_csv("verify_sample.csv")
###########################################################

###########################################################
# 提交的代码
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

CV_NUM=3

###########################################################
model_sample = model_sample.fillna(-1)
verify_sample = verify_sample.fillna(-1)

features_col = [c for c in model_sample.columns if c not in ['user_id','y']]
X = model_sample[features_col].values
y = model_sample['y'].values
X_pred = verify_sample[features_col].values


skf = StratifiedKFold(n_splits=CV_NUM, shuffle=True, random_state=1)

params = {
		'task': 'train',
		'boosting_type': 'gbdt',
		'objective': 'binary',
		'metric': 'auc',
		'num_leaves': 9,
		'learning_rate': 0.03,
		'feature_fraction_seed': 2,
		'feature_fraction': 0.9,
		'bagging_fraction': 0.8,
		'bagging_freq': 5,
		'min_data': 20,
		'min_hessian': 1,
		'verbose': -1,
		}


layer_train = np.zeros((X.shape[0], CV_NUM))
test_pred = np.zeros((X_pred.shape[0], CV_NUM))

for k,(train_index, test_index) in enumerate(skf.split(X, y)):
	X_train = X[train_index]
	y_train = y[train_index]
	X_test = X[test_index]
	y_test = y[test_index]

	# create dataset for lightgbm
	lgb_train = lgb.Dataset(X_train, y_train)
	lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

	# train
	gbm = lgb.train(params,
				lgb_train,
				num_boost_round=10000,
				valid_sets=lgb_eval,
				early_stopping_rounds=200)

	pred_y = gbm.predict(X_test, num_iteration=gbm.best_iteration)
	layer_train[test_index, 1] = pred_y

	pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
	test_pred[:, k] = pred
layer_test = test_pred.mean(axis=1)


X = pd.DataFrame(X)
X['pre']=layer_train[:,1]
X_pred = pd.DataFrame(X_pred)
X_pred['pre'] = layer_test

X = X.values
X_pred = X_pred.values

for cv in range(CV_NUM):
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=cv)

# for cv,(train_index, test_index) in enumerate(skf.split(X, y)):
#	X_train = X[train_index]
#	y_train = y[train_index]
#	X_test = X[test_index]
#	y_test = y[test_index]
	
	# create dataset for lightgbm
	lgb_train = lgb.Dataset(X_train, y_train)
	lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

	# specify your configurations as a dict
	params = {
			'task': 'train',
			'boosting_type': 'gbdt',
			'objective': 'binary',
			'metric': 'auc',
			'num_leaves': 9,
			'learning_rate': 0.03,
			'feature_fraction_seed': 2,
			'feature_fraction': 0.9,
			'bagging_fraction': 0.8,
			'bagging_freq': 5,
			'min_data': 20,
			'min_hessian': 1,
			'verbose': -1,
			}

	# train
	gbm = lgb.train(params,
				lgb_train,
				num_boost_round=10000,
				valid_sets=lgb_eval,
				early_stopping_rounds=200)

	# print('Save model...')
	# save model to file
	# gbm.save_model('model.txt')

	# print('Start predicting...')
	# predict
	# y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
	# eval
	# print('The f1 of prediction is:', f1_score(y_test, np.where(y_pred >= 0.225,1,0)))

	pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
	if cv == 0:
		pred_out=pred
	else:
		pred_out+=pred


pred_out = pred_out * 1.0 / CV_NUM
pred = np.where(pred_out >= 0.24,1,0)
verify_sample['y_prediction'] = pred
predict_result = verify_sample[['user_id','y_prediction']]
###########################################################

###########################################################
# 非官方部分，线下测试
from sklearn.metrics import f1_score
print(f1_score(verify_sample['y'],pred))
###########################################################

###########################################################
# 官方部分
# f1_score = score(predict_result)
###########################################################


# SBBTree

In [ ]:
# -*- coding: utf-8 -*-
###########################################################
# 提交的代码
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

class SBBTree():
	"""Stacking,Bootstap,Bagging----SBBTree"""
	""" author：Cookly """
	def __init__(self, params, stacking_num, bagging_num, bagging_test_size, num_boost_round, early_stopping_rounds):
		"""
		Initializes the SBBTree.
        Args:
          params : lgb params.
          stacking_num : k_flod stacking.
          bagging_num : bootstrap num.
          bagging_test_size : bootstrap sample rate.
          num_boost_round : boost num.
		  early_stopping_rounds : early_stopping_rounds.
        """
		self.params = params
		self.stacking_num = stacking_num
		self.bagging_num = bagging_num
		self.bagging_test_size = bagging_test_size
		self.num_boost_round = num_boost_round
		self.early_stopping_rounds = early_stopping_rounds

		self.model = lgb
		self.stacking_model = []
		self.bagging_model = []

	def fit(self, X, y):
		""" fit model. """
		if self.stacking_num > 1:
			layer_train = np.zeros((X.shape[0], 2))
			self.SK = StratifiedKFold(n_splits=self.stacking_num, shuffle=True, random_state=1)
			for k,(train_index, test_index) in enumerate(self.SK.split(X, y)):
				X_train = X[train_index]
				y_train = y[train_index]
				X_test = X[test_index]
				y_test = y[test_index]

				lgb_train = lgb.Dataset(X_train, y_train)
				lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

				gbm = lgb.train(self.params,
							lgb_train,
							num_boost_round=self.num_boost_round,
							valid_sets=lgb_eval,
							early_stopping_rounds=self.early_stopping_rounds)

				self.stacking_model.append(gbm)

				pred_y = gbm.predict(X_test, num_iteration=gbm.best_iteration)
				layer_train[test_index, 1] = pred_y

			X = np.hstack((X, layer_train[:,1].reshape((-1,1)))) 
		else:
			pass
		for bn in range(self.bagging_num):
			X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.bagging_test_size, random_state=bn)
	
			lgb_train = lgb.Dataset(X_train, y_train)
			lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

			gbm = lgb.train(params,
						lgb_train,
						num_boost_round=10000,
						valid_sets=lgb_eval,
						early_stopping_rounds=200)

			self.bagging_model.append(gbm)
		
	def predict(self, X_pred):
		""" predict test data. """
		if self.stacking_num > 1:
			test_pred = np.zeros((X_pred.shape[0], self.stacking_num))
			for sn,gbm in enumerate(self.stacking_model):
				pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
				test_pred[:, sn] = pred
			X_pred = np.hstack((X_pred, test_pred.mean(axis=1).reshape((-1,1))))  
		else:
			pass 
		for bn,gbm in enumerate(self.bagging_model):
			pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
			if bn == 0:
				pred_out=pred
			else:
				pred_out+=pred
		return pred_out/self.bagging_num

###########################################################################
# test code
from sklearn.datasets import make_classification
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import make_gaussian_quantiles
from sklearn import metrics
from sklearn.metrics import f1_score
# X, y = make_classification(n_samples=1000, n_features=25, n_clusters_per_class=1, n_informative=15, random_state=1)
X, y = make_gaussian_quantiles(mean=None, cov=1.0, n_samples=1000, n_features=50, n_classes=2, shuffle=True, random_state=2)
# data = load_breast_cancer()
# X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
params = {
		'task': 'train',
		'boosting_type': 'gbdt',
		'objective': 'binary',
		'metric': 'auc',
		'num_leaves': 9,
		'learning_rate': 0.03,
		'feature_fraction_seed': 2,
		'feature_fraction': 0.9,
		'bagging_fraction': 0.8,
		'bagging_freq': 5,
		'min_data': 20,
		'min_hessian': 1,
		'verbose': -1,
		'silent': 0
		}
# test 1
model = SBBTree(params=params, stacking_num=2, bagging_num=1,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X,y)
X_pred = X[0].reshape((1,-1))
pred=model.predict(X_pred)
print('pred')
print(pred)
print('TEST 1 ok')

'''
# test 1
model = SBBTree(params, stacking_num=1, bagging_num=1, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred1=model.predict(X_test)
# test 2 
model = SBBTree(params, stacking_num=1, bagging_num=3, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred2=model.predict(X_test)
# test 3 
model = SBBTree(params, stacking_num=5, bagging_num=1, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred3=model.predict(X_test)
# test 4 
model = SBBTree(params, stacking_num=5, bagging_num=3, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
model.fit(X_train,y_train)
pred4=model.predict(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred1, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred2, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred3, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))
fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred4, pos_label=2)
print('auc: ',metrics.auc(fpr, tpr))
'''

# 深度stacking

In [114]:
inputs = tf.keras.Input(shape=(60,8))
    
# 共享层    
layer_cnn = tf.keras.layers.Conv1D(64, 10, padding='same')(inputs)
layer_maxpool = tf.keras.layers.MaxPool1D(6, padding='same')(layer_cnn)
    
for i in [3,3,3,3,3]:
    layer_cnn = tf.keras.layers.Conv1D(64, i, padding='same')(layer_maxpool)
    layer_maxpool = tf.keras.layers.MaxPool1D(2, padding='same')(layer_cnn)

layer_flatten = tf.keras.layers.Flatten()(layer_maxpool)


# 场景 + 动作
layer_dense_cls = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(layer_flatten)
outputs_cls = tf.keras.layers.Dense(units=19, activation=tf.nn.softmax)(layer_dense_cls)

# 场景
layer_dense_sence = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(layer_flatten)
outputs_sence = tf.keras.layers.Dense(units=4, activation=tf.nn.softmax)(layer_dense_sence)

# 动作
layer_dense_action = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(layer_flatten)
outputs_action = tf.keras.layers.Dense(units=7, activation=tf.nn.softmax)(layer_dense_action)


# stacking层
list_layer = [
              layer_flatten, 
              layer_dense_cls, 
              # outputs_cls, 
              layer_dense_sence, 
              # outputs_sence,
              layer_dense_action,
              # outputs_action
             ]
layer_stacking = tf.concat(list_layer,axis=-1)
outputs_fin = tf.keras.layers.Dense(units=19, activation=tf.nn.softmax)(layer_dense_cls)

outputs = [outputs_cls,outputs_sence,outputs_action,outputs_fin]

model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['acc'])

In [115]:
model.fit(x[xx],[y_cls_[xx],y_sence_[xx],y_action_[xx],y_cls_[xx]],
         validation_data=[x[yy], [y_cls_[yy],y_sence_[yy],y_action_[yy],y_cls_[yy]]]
         )

Train on 5833 samples, validate on 1459 samples
5833/5833 [==============================] - 6s 1ms/sample - loss: 6.6612 - dense_148_loss: 2.2203 - dense_150_loss: 0.8300 - dense_152_loss: 1.3592 - dense_153_loss: 2.2433 - dense_148_acc: 0.2848 - dense_150_acc: 0.6029 - dense_152_acc: 0.4879 - dense_153_acc: 0.2789 - val_loss: 5.6458 - val_dense_148_loss: 1.8662 - val_dense_150_loss: 0.6815 - val_dense_152_loss: 1.2360 - val_dense_153_loss: 1.8558 - val_dense_148_acc: 0.3715 - val_dense_150_acc: 0.6854 - val_dense_152_acc: 0.5147 - val_dense_153_acc: 0.3804
